# Pandas Multiple or Heirarchical Indexing

In [2]:
import pandas as pd 
import numpy as np 

In [16]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [17]:
index = pd.MultiIndex.from_tuples(index)

In [18]:
pop.reindex(index)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [20]:
pop = pd.Series(populations, index=index)

In [21]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [22]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


> Convert Multiply indexed `Series` into a conventionally indexed `DataFrame`.

In [23]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

>Add a column

In [24]:
pop_df = pd.DataFrame({
    'total': pop,
    'under18': [9284094, 8898092,
                4318033, 4181528,
                6879014, 7432474]
})
pop_df

total  under18
California 2000  33871648  9284094
           2010  37253956  8898092
New York   2000  18976457  4318033
           2010  19378102  4181528
Texas      2000  20851820  6879014
           2010  25145561  7432474

In [25]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.274096
            2010    0.238850
New York    2000    0.227547
            2010    0.215786
Texas       2000    0.329900
            2010    0.295578
dtype: float64

In [26]:
f_u18.unstack()

,2000,2010
California,0.274096,0.238850
New York,0.227547,0.215786
Texas,0.329900,0.295578


# Multiindex Creation

In [27]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a','a','b','b'],[1,2,1,2]], 
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.653224  0.249848
  2  0.333277  0.108866
b 1  0.250907  0.088180
  2  0.304413  0.792930

> A dictionary with tuples as keys

In [28]:
data = {('California', 2010): 37253956,
        ('California', 2020): 39538223,
        ('New York', 2010): 19378102,
        ('New York', 2020): 20201249,
        ('Texas', 2010): 25145561,
        ('Texas', 2020): 29145505}
pd.Series(data)

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

>Explicit multiindex constructors

In [29]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [30]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [31]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [32]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

>Level Names

In [33]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [34]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Multiindex for Columns

In [35]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

In [42]:
data = np.round(np.random.randn(4,6), 1)
data[:,::2] *= 10

data += 37
data

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  37.9  29.0  38.2  34.0  35.3
     2      49.0  36.7  38.0  38.6  29.0  36.2
2014 1      48.0  35.2  33.0  39.0  34.0  38.9
     2      38.0  38.1  51.0  38.6  30.0  38.2

In [43]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      29.0  38.2
     2      38.0  38.6
2014 1      33.0  39.0
     2      51.0  38.6

In [50]:
health_data.loc[2013]

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
visit                                      
1        27.0  37.9  29.0  38.2  34.0  35.3
2        49.0  36.7  38.0  38.6  29.0  36.2

In [51]:
health_data['Guido','HR']

year  visit
2013  1        29.0
      2        38.0
2014  1        33.0
      2        51.0
Name: (Guido, HR), dtype: float64

In [58]:
health_data.iloc[:2,1:2]

subject      Bob
type        Temp
year visit      
2013 1      37.9
     2      36.7

In [59]:
health_data.iloc[:2]

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  37.9  29.0  38.2  34.0  35.3
     2      49.0  36.7  38.0  38.6  29.0  36.2

In [62]:
health_data.iloc[:2,2:3]

subject    Guido
type          HR
year visit      
2013 1      29.0
     2      38.0

In [63]:
# Fail 
# health_data.loc[(:, 1), (:, 'HR')]

In [64]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,27.0,29.0,34.0
2014,1,48.0,33.0,34.0


## Rearranging multi-indexes

In [65]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.372084
      2      0.592588
c     1      0.112209
      2      0.565792
b     1      0.464795
      2      0.449224
dtype: float64

In [66]:
data.sort_index()

char  int
a     1      0.372084
      2      0.592588
b     1      0.464795
      2      0.449224
c     1      0.112209
      2      0.565792
dtype: float64

In [67]:
data.sort_index()['a':'b']

char  int
a     1      0.372084
      2      0.592588
b     1      0.464795
      2      0.449224
dtype: float64

## Stacking and unstacking

In [68]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [69]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [70]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

# Index setting and resetting

In [71]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [72]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [78]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561